<a href="https://colab.research.google.com/github/mcgmed/SQL/blob/main/Hospital_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect('hospital.db')
cur = con.cursor()

In [2]:
cur.execute('''CREATE TABLE Physician (EmployeeID INTEGER PRIMARY KEY NOT NULL,
                                       Name TEXT NOT NULL,
                                       Position TEXT NOT NULL,
                                       SSN INTEGER NOT NULL)''')

cur.execute('''CREATE TABLE Department (DepartmentID INTEGER PRIMARY KEY NOT NULL,
                                        Name TEXT NOT NULL,
                                        Head INTEGER NOT NULL,
                                        FOREIGN KEY (Head) REFERENCES Physician(EmployeeID))''') 

cur.execute('''CREATE TABLE Affiliated_With (Physician INTEGER NOT NULL,
                                             Department INTEGER NOT NULL,
                                             PrimaryAffiliation BOOLEAN NOT NULL,
                                             PRIMARY KEY(Physician, Department),
                                             FOREIGN KEY (Physician) REFERENCES Physician(EmployeeID),
                                             FOREIGN KEY (Department) REFERENCES Department(DepartmentID))''')

cur.execute('''CREATE TABLE Procedure (Code INTEGER PRIMARY KEY NOT NULL,
                                       Name TEXT NOT NULL,
                                       Cost REAL NOT NULL)''')

cur.execute('''CREATE TABLE Trained_In (Physician INTEGER NOT NULL,
                                        Treatment INTEGER NOT NULL,
                                        CertificationDate DATETIME NOT NULL,
                                        CertificationExpires DATETIME NOT NULL,
                                        PRIMARY KEY(Physician, Treatment),
                                        FOREIGN KEY (Physician) REFERENCES Physician(EmployeeID),
                                        FOREIGN KEY (Treatment) REFERENCES Procedure(Code))''')

cur.execute('''CREATE TABLE Patient (SSN INTEGER PRIMARY KEY NOT NULL,
                                     Name TEXT NOT NULL,
                                     Address TEXT NOT NULL,
                                     Phone TEXT NOT NULL,
                                     InsuranceID INTEGER NOT NULL,
                                     PCP INTEGER NOT NULL,
                                     FOREIGN KEY (PCP) REFERENCES Physician(EmployeeID))''')

cur.execute('''CREATE TABLE Nurse (EmployeeID INTEGER PRIMARY KEY NOT NULL,
                                   Name TEXT NOT NULL,
                                   Position TEXT NOT NULL,
                                   Registered BOOLEAN NOT NULL,
                                   SSN INTEGER NOT NULL)''')

cur.execute('''CREATE TABLE Appointment (AppointmentID INTEGER PRIMARY KEY NOT NULL,
                                         Patient INTEGER NOT NULL,
                                         PrepNurse INTEGER,
                                         Physician INTEGER NOT NULL,
                                         Start DATETIME NOT NULL,
                                         End DATETIME NOT NULL,
                                         ExaminationRoom TEXT NOT NULL,
                                         FOREIGN KEY (Patient) REFERENCES Patient(SSN),
                                         FOREIGN KEY (PrepNurse) REFERENCES Nurse(EmployeeID),
                                         FOREIGN KEY (Physician) REFERENCES Physician(EmployeeID))''')

cur.execute('''CREATE TABLE Medication (Code INTEGER PRIMARY KEY NOT NULL,
                                        Name TEXT NOT NULL,
                                        Brand TEXT NOT NULL,
                                        Description TEXT NOT NULL)''')

cur.execute('''CREATE TABLE Prescribes (Physician INTEGER NOT NULL,
                                        Patient INTEGER NOT NULL,
                                        Medication INTEGER NOT NULL,
                                        Date DATETIME NOT NULL,
                                        Appointment INTEGER,
                                        Dose TEXT NOT NULL,
                                        PRIMARY KEY(Physician, Patient, Medication, Date),
                                        FOREIGN KEY (Physician) REFERENCES Physician(EmployeeID),
                                        FOREIGN KEY (Patient) REFERENCES Patient(SSN),
                                        FOREIGN KEY (Medication) REFERENCES Medication(Code),
                                        FOREIGN KEY (Appointment) REFERENCES Appointment(AppointmentID))''')

cur.execute('''CREATE TABLE Block (Floor INTEGER NOT NULL,
                                   Code INTEGER NOT NULL,
                                   PRIMARY KEY(Floor, Code))''')

cur.execute('''CREATE TABLE Room (Number INTEGER PRIMARY KEY NOT NULL,
                                  Type TEXT NOT NULL,
                                  BlockFloor INTEGER NOT NULL,
                                  BlockCode INTEGER NOT NULL,
                                  Unavailable BOOLEAN NOT NULL,
                                  FOREIGN KEY(BlockFloor, BlockCode) REFERENCES Block)''')

cur.execute('''CREATE TABLE On_Call (Nurse INTEGER NOT NULL,
                                     BlockFloor INTEGER NOT NULL,
                                     BlockCode INTEGER NOT NULL,
                                     Start DATETIME NOT NULL,
                                     End DATETIME NOT NULL,
                                     PRIMARY KEY (Nurse, BlockFloor, BlockCode, Start, End),
                                     FOREIGN KEY (BlockFloor, BlockCode) REFERENCES Block,
                                     FOREIGN KEY (Nurse) REFERENCES Nurse(EmployeeID))''')

cur.execute('''CREATE TABLE Stay (StayID INTEGER PRIMARY KEY NOT NULL,
                                  Patient INTEGER NOT NULL,
                                  Room INTEGER NOT NULL,
                                  Start DATETIME NOT NULL,
                                  End DATETIME NOT NULL,
                                  FOREIGN KEY (Patient) REFERENCES Patient(SSN),
                                  FOREIGN KEY (Room) REFERENCES Room(Number))''')

cur.execute('''CREATE TABLE Undergoes (Patient INTEGER NOT NULL,
                                       Procedure INTEGER NOT NULL,
                                       Stay INTEGER NOT NULL,
                                       Date DATETIME NOT NULL,
                                       Physician INTEGER NOT NULL,
                                       AssistingNurse INTEGER,
                                       PRIMARY KEY (Patient, Procedure, Stay, Date),
                                       FOREIGN KEY (Patient) REFERENCES Patient(SSN),
                                       FOREIGN KEY (Procedure) REFERENCES Procedure(Code),
                                       FOREIGN KEY (Stay) REFERENCES Stay(StayID),
                                       FOREIGN KEY (Physician) REFERENCES Physician(EmployeeID),
                                       FOREIGN KEY (AssistingNurse) REFERENCES Nurse(EmployeeID))''')

con.commit()

In [3]:
cur.execute("INSERT INTO Physician VALUES(1,'John Dorian','Staff Internist',111111111);")
cur.execute("INSERT INTO Physician VALUES(2,'Elliot Reid','Attending Physician',222222222);")
cur.execute("INSERT INTO Physician VALUES(3,'Christopher Turk','Surgical Attending Physician',333333333);")
cur.execute("INSERT INTO Physician VALUES(4,'Percival Cox','Senior Attending Physician',444444444);")
cur.execute("INSERT INTO Physician VALUES(5,'Bob Kelso','Head Chief of Medicine',555555555);")
cur.execute("INSERT INTO Physician VALUES(6,'Todd Quinlan','Surgical Attending Physician',666666666);")
cur.execute("INSERT INTO Physician VALUES(7,'John Wen','Surgical Attending Physician',777777777);")
cur.execute("INSERT INTO Physician VALUES(8,'Keith Dudemeister','MD Resident',888888888);")
cur.execute("INSERT INTO Physician VALUES(9,'Molly Clock','Attending Psychiatrist',999999999);")

cur.execute("INSERT INTO Department VALUES(1,'General Medicine',4);")
cur.execute("INSERT INTO Department VALUES(2,'Surgery',7);")
cur.execute("INSERT INTO Department VALUES(3,'Psychiatry',9);")

cur.execute("INSERT INTO Affiliated_With VALUES(1,1,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(2,1,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(3,1,0);")
cur.execute("INSERT INTO Affiliated_With VALUES(3,2,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(4,1,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(5,1,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(6,2,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(7,1,0);")
cur.execute("INSERT INTO Affiliated_With VALUES(7,2,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(8,1,1);")
cur.execute("INSERT INTO Affiliated_With VALUES(9,3,1);")

cur.execute("INSERT INTO Procedure VALUES(1,'Reverse Rhinopodoplasty',1500.0);")
cur.execute("INSERT INTO Procedure VALUES(2,'Obtuse Pyloric Recombobulation',3750.0);")
cur.execute("INSERT INTO Procedure VALUES(3,'Folded Demiophtalmectomy',4500.0);")
cur.execute("INSERT INTO Procedure VALUES(4,'Complete Walletectomy',10000.0);")
cur.execute("INSERT INTO Procedure VALUES(5,'Obfuscated Dermogastrotomy',4899.0);")
cur.execute("INSERT INTO Procedure VALUES(6,'Reversible Pancreomyoplasty',5600.0);")
cur.execute("INSERT INTO Procedure VALUES(7,'Follicular Demiectomy',25.0);")

cur.execute("INSERT INTO Patient VALUES(100000001,'John Smith','42 Foobar Lane','555-0256',68476213,1);")
cur.execute("INSERT INTO Patient VALUES(100000002,'Grace Ritchie','37 Snafu Drive','555-0512',36546321,2);")
cur.execute("INSERT INTO Patient VALUES(100000003,'Random J. Patient','101 Omgbbq Street','555-1204',65465421,2);")
cur.execute("INSERT INTO Patient VALUES(100000004,'Dennis Doe','1100 Foobaz Avenue','555-2048',68421879,3);")

cur.execute("INSERT INTO Nurse VALUES(101,'Carla Espinosa','Head Nurse',1,111111110);")
cur.execute("INSERT INTO Nurse VALUES(102,'Laverne Roberts','Nurse',1,222222220);")
cur.execute("INSERT INTO Nurse VALUES(103,'Paul Flowers','Nurse',0,333333330);")

cur.execute("INSERT INTO Appointment VALUES(13216584,100000001,101,1,'2008-04-24 10:00','2008-04-24 11:00','A');")
cur.execute("INSERT INTO Appointment VALUES(26548913,100000002,101,2,'2008-04-24 10:00','2008-04-24 11:00','B');")
cur.execute("INSERT INTO Appointment VALUES(36549879,100000001,102,1,'2008-04-25 10:00','2008-04-25 11:00','A');")
cur.execute("INSERT INTO Appointment VALUES(46846589,100000004,103,4,'2008-04-25 10:00','2008-04-25 11:00','B');")
cur.execute("INSERT INTO Appointment VALUES(59871321,100000004,NULL,4,'2008-04-26 10:00','2008-04-26 11:00','C');")
cur.execute("INSERT INTO Appointment VALUES(69879231,100000003,103,2,'2008-04-26 11:00','2008-04-26 12:00','C');")
cur.execute("INSERT INTO Appointment VALUES(76983231,100000001,NULL,3,'2008-04-26 12:00','2008-04-26 13:00','C');")
cur.execute("INSERT INTO Appointment VALUES(86213939,100000004,102,9,'2008-04-27 10:00','2008-04-21 11:00','A');")
cur.execute("INSERT INTO Appointment VALUES(93216548,100000002,101,2,'2008-04-27 10:00','2008-04-27 11:00','B');")

cur.execute("INSERT INTO Medication VALUES(1,'Procrastin-X','X','N/A');")
cur.execute("INSERT INTO Medication VALUES(2,'Thesisin','Foo Labs','N/A');")
cur.execute("INSERT INTO Medication VALUES(3,'Awakin','Bar Laboratories','N/A');")
cur.execute("INSERT INTO Medication VALUES(4,'Crescavitin','Baz Industries','N/A');")
cur.execute("INSERT INTO Medication VALUES(5,'Melioraurin','Snafu Pharmaceuticals','N/A');")

cur.execute("INSERT INTO Prescribes VALUES(1,100000001,1,'2008-04-24 10:47',13216584,'5');")
cur.execute("INSERT INTO Prescribes VALUES(9,100000004,2,'2008-04-27 10:53',86213939,'10');")
cur.execute("INSERT INTO Prescribes VALUES(9,100000004,2,'2008-04-30 16:53',NULL,'5');")

cur.execute("INSERT INTO Block VALUES(1,1);")
cur.execute("INSERT INTO Block VALUES(1,2);")
cur.execute("INSERT INTO Block VALUES(1,3);")
cur.execute("INSERT INTO Block VALUES(2,1);")
cur.execute("INSERT INTO Block VALUES(2,2);")
cur.execute("INSERT INTO Block VALUES(2,3);")
cur.execute("INSERT INTO Block VALUES(3,1);")
cur.execute("INSERT INTO Block VALUES(3,2);")
cur.execute("INSERT INTO Block VALUES(3,3);")
cur.execute("INSERT INTO Block VALUES(4,1);")
cur.execute("INSERT INTO Block VALUES(4,2);")
cur.execute("INSERT INTO Block VALUES(4,3);")

cur.execute("INSERT INTO Room VALUES(101,'Single',1,1,0);")
cur.execute("INSERT INTO Room VALUES(102,'Single',1,1,0);")
cur.execute("INSERT INTO Room VALUES(103,'Single',1,1,0);")
cur.execute("INSERT INTO Room VALUES(111,'Single',1,2,0);")
cur.execute("INSERT INTO Room VALUES(112,'Single',1,2,1);")
cur.execute("INSERT INTO Room VALUES(113,'Single',1,2,0);")
cur.execute("INSERT INTO Room VALUES(121,'Single',1,3,0);")
cur.execute("INSERT INTO Room VALUES(122,'Single',1,3,0);")
cur.execute("INSERT INTO Room VALUES(123,'Single',1,3,0);")
cur.execute("INSERT INTO Room VALUES(201,'Single',2,1,1);")
cur.execute("INSERT INTO Room VALUES(202,'Single',2,1,0);")
cur.execute("INSERT INTO Room VALUES(203,'Single',2,1,0);")
cur.execute("INSERT INTO Room VALUES(211,'Single',2,2,0);")
cur.execute("INSERT INTO Room VALUES(212,'Single',2,2,0);")
cur.execute("INSERT INTO Room VALUES(213,'Single',2,2,1);")
cur.execute("INSERT INTO Room VALUES(221,'Single',2,3,0);")
cur.execute("INSERT INTO Room VALUES(222,'Single',2,3,0);")
cur.execute("INSERT INTO Room VALUES(223,'Single',2,3,0);")
cur.execute("INSERT INTO Room VALUES(301,'Single',3,1,0);")
cur.execute("INSERT INTO Room VALUES(302,'Single',3,1,1);")
cur.execute("INSERT INTO Room VALUES(303,'Single',3,1,0);")
cur.execute("INSERT INTO Room VALUES(311,'Single',3,2,0);")
cur.execute("INSERT INTO Room VALUES(312,'Single',3,2,0);")
cur.execute("INSERT INTO Room VALUES(313,'Single',3,2,0);")
cur.execute("INSERT INTO Room VALUES(321,'Single',3,3,1);")
cur.execute("INSERT INTO Room VALUES(322,'Single',3,3,0);")
cur.execute("INSERT INTO Room VALUES(323,'Single',3,3,0);")
cur.execute("INSERT INTO Room VALUES(401,'Single',4,1,0);")
cur.execute("INSERT INTO Room VALUES(402,'Single',4,1,1);")
cur.execute("INSERT INTO Room VALUES(403,'Single',4,1,0);")
cur.execute("INSERT INTO Room VALUES(411,'Single',4,2,0);")
cur.execute("INSERT INTO Room VALUES(412,'Single',4,2,0);")
cur.execute("INSERT INTO Room VALUES(413,'Single',4,2,0);")
cur.execute("INSERT INTO Room VALUES(421,'Single',4,3,1);")
cur.execute("INSERT INTO Room VALUES(422,'Single',4,3,0);")
cur.execute("INSERT INTO Room VALUES(423,'Single',4,3,0);")

cur.execute("INSERT INTO On_Call VALUES(101,1,1,'2008-11-04 11:00','2008-11-04 19:00');")
cur.execute("INSERT INTO On_Call VALUES(101,1,2,'2008-11-04 11:00','2008-11-04 19:00');")
cur.execute("INSERT INTO On_Call VALUES(102,1,3,'2008-11-04 11:00','2008-11-04 19:00');")
cur.execute("INSERT INTO On_Call VALUES(103,1,1,'2008-11-04 19:00','2008-11-05 03:00');")
cur.execute("INSERT INTO On_Call VALUES(103,1,2,'2008-11-04 19:00','2008-11-05 03:00');")
cur.execute("INSERT INTO On_Call VALUES(103,1,3,'2008-11-04 19:00','2008-11-05 03:00');")

cur.execute("INSERT INTO Stay VALUES(3215,100000001,111,'2008-05-01','2008-05-04');")
cur.execute("INSERT INTO Stay VALUES(3216,100000003,123,'2008-05-03','2008-05-14');")
cur.execute("INSERT INTO Stay VALUES(3217,100000004,112,'2008-05-02','2008-05-03');")

cur.execute("INSERT INTO Undergoes VALUES(100000001,6,3215,'2008-05-02',3,101);")
cur.execute("INSERT INTO Undergoes VALUES(100000001,2,3215,'2008-05-03',7,101);")
cur.execute("INSERT INTO Undergoes VALUES(100000004,1,3217,'2008-05-07',3,102);")
cur.execute("INSERT INTO Undergoes VALUES(100000004,5,3217,'2008-05-09',6,NULL);")
cur.execute("INSERT INTO Undergoes VALUES(100000001,7,3217,'2008-05-10',7,101);")
cur.execute("INSERT INTO Undergoes VALUES(100000004,4,3217,'2008-05-13',3,103);")

cur.execute("INSERT INTO Trained_In VALUES(3,1,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(3,2,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(3,5,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(3,6,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(3,7,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(6,2,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(6,5,'2007-01-01','2007-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(6,6,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,1,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,2,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,3,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,4,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,5,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,6,'2008-01-01','2008-12-31');")
cur.execute("INSERT INTO Trained_In VALUES(7,7,'2008-01-01','2008-12-31');")

con.commit()

In [4]:
query = 'SELECT * FROM Physician'
data = pd.read_sql(query, con)
data

,EmployeeID,Name,Position,SSN
0,1,John Dorian,Staff Internist,111111111
1,2,Elliot Reid,Attending Physician,222222222
2,3,Christopher Turk,Surgical Attending Physician,333333333
3,4,Percival Cox,Senior Attending Physician,444444444
4,5,Bob Kelso,Head Chief of Medicine,555555555
5,6,Todd Quinlan,Surgical Attending Physician,666666666
6,7,John Wen,Surgical Attending Physician,777777777
7,8,Keith Dudemeister,MD Resident,888888888
8,9,Molly Clock,Attending Psychiatrist,999999999


In [5]:
query = 'SELECT * FROM Department'
data = pd.read_sql(query, con)
data

,DepartmentID,Name,Head
0,1,General Medicine,4
1,2,Surgery,7
2,3,Psychiatry,9


In [6]:
query = 'SELECT * FROM Affiliated_With'
data = pd.read_sql(query, con)
data

,Physician,Department,PrimaryAffiliation
0,1,1,1
1,2,1,1
2,3,1,0
3,3,2,1
4,4,1,1
5,5,1,1
6,6,2,1
7,7,1,0
8,7,2,1
9,8,1,1


In [7]:
query = 'SELECT * FROM Procedure'
data = pd.read_sql(query, con)
data

,Code,Name,Cost
0,1,Reverse Rhinopodoplasty,1500.0
1,2,Obtuse Pyloric Recombobulation,3750.0
2,3,Folded Demiophtalmectomy,4500.0
3,4,Complete Walletectomy,10000.0
4,5,Obfuscated Dermogastrotomy,4899.0
5,6,Reversible Pancreomyoplasty,5600.0
6,7,Follicular Demiectomy,25.0


In [8]:
query = 'SELECT * FROM Trained_In'
data = pd.read_sql(query, con)
data

,Physician,Treatment,CertificationDate,CertificationExpires
0,3,1,2008-01-01,2008-12-31
1,3,2,2008-01-01,2008-12-31
2,3,5,2008-01-01,2008-12-31
3,3,6,2008-01-01,2008-12-31
4,3,7,2008-01-01,2008-12-31
5,6,2,2008-01-01,2008-12-31
6,6,5,2007-01-01,2007-12-31
7,6,6,2008-01-01,2008-12-31
8,7,1,2008-01-01,2008-12-31
9,7,2,2008-01-01,2008-12-31


In [9]:
query = 'SELECT * FROM Patient'
data = pd.read_sql(query, con)
data

,SSN,Name,Address,Phone,InsuranceID,PCP
0,100000001,John Smith,42 Foobar Lane,555-0256,68476213,1
1,100000002,Grace Ritchie,37 Snafu Drive,555-0512,36546321,2
2,100000003,Random J. Patient,101 Omgbbq Street,555-1204,65465421,2
3,100000004,Dennis Doe,1100 Foobaz Avenue,555-2048,68421879,3


In [10]:
query = 'SELECT * FROM Nurse'
data = pd.read_sql(query, con)
data

,EmployeeID,Name,Position,Registered,SSN
0,101,Carla Espinosa,Head Nurse,1,111111110
1,102,Laverne Roberts,Nurse,1,222222220
2,103,Paul Flowers,Nurse,0,333333330


In [11]:
query = 'SELECT * FROM Appointment'
data = pd.read_sql(query, con)
data

,AppointmentID,Patient,PrepNurse,Physician,Start,End,ExaminationRoom
0,13216584,100000001,101.0,1,2008-04-24 10:00,2008-04-24 11:00,A
1,26548913,100000002,101.0,2,2008-04-24 10:00,2008-04-24 11:00,B
2,36549879,100000001,102.0,1,2008-04-25 10:00,2008-04-25 11:00,A
3,46846589,100000004,103.0,4,2008-04-25 10:00,2008-04-25 11:00,B
4,59871321,100000004,NaN,4,2008-04-26 10:00,2008-04-26 11:00,C
5,69879231,100000003,103.0,2,2008-04-26 11:00,2008-04-26 12:00,C
6,76983231,100000001,NaN,3,2008-04-26 12:00,2008-04-26 13:00,C
7,86213939,100000004,102.0,9,2008-04-27 10:00,2008-04-21 11:00,A
8,93216548,100000002,101.0,2,2008-04-27 10:00,2008-04-27 11:00,B


In [12]:
query = 'SELECT * FROM Medication'
data = pd.read_sql(query, con)
data

,Code,Name,Brand,Description
0,1,Procrastin-X,X,N/A
1,2,Thesisin,Foo Labs,N/A
2,3,Awakin,Bar Laboratories,N/A
3,4,Crescavitin,Baz Industries,N/A
4,5,Melioraurin,Snafu Pharmaceuticals,N/A


In [13]:
query = 'SELECT * FROM Prescribes'
data = pd.read_sql(query, con)
data

,Physician,Patient,Medication,Date,Appointment,Dose
0,1,100000001,1,2008-04-24 10:47,13216584.0,5
1,9,100000004,2,2008-04-27 10:53,86213939.0,10
2,9,100000004,2,2008-04-30 16:53,NaN,5


In [14]:
query = 'SELECT * FROM Block'
data = pd.read_sql(query, con)
data

,Floor,Code
0,1,1
1,1,2
2,1,3
3,2,1
4,2,2
5,2,3
6,3,1
7,3,2
8,3,3
9,4,1


In [15]:
query = 'SELECT * FROM Room'
data = pd.read_sql(query, con)
data

,Number,Type,BlockFloor,BlockCode,Unavailable
0,101,Single,1,1,0
1,102,Single,1,1,0
2,103,Single,1,1,0
3,111,Single,1,2,0
4,112,Single,1,2,1
5,113,Single,1,2,0
6,121,Single,1,3,0
7,122,Single,1,3,0
8,123,Single,1,3,0
9,201,Single,2,1,1


In [16]:
query = 'SELECT * FROM On_Call'
data = pd.read_sql(query, con)
data

,Nurse,BlockFloor,BlockCode,Start,End
0,101,1,1,2008-11-04 11:00,2008-11-04 19:00
1,101,1,2,2008-11-04 11:00,2008-11-04 19:00
2,102,1,3,2008-11-04 11:00,2008-11-04 19:00
3,103,1,1,2008-11-04 19:00,2008-11-05 03:00
4,103,1,2,2008-11-04 19:00,2008-11-05 03:00
5,103,1,3,2008-11-04 19:00,2008-11-05 03:00


In [17]:
query = 'SELECT * FROM Stay'
data = pd.read_sql(query, con)
data

,StayID,Patient,Room,Start,End
0,3215,100000001,111,2008-05-01,2008-05-04
1,3216,100000003,123,2008-05-03,2008-05-14
2,3217,100000004,112,2008-05-02,2008-05-03


In [18]:
query = 'SELECT * FROM Undergoes'
data = pd.read_sql(query, con)
data

,Patient,Procedure,Stay,Date,Physician,AssistingNurse
0,100000001,6,3215,2008-05-02,3,101.0
1,100000001,2,3215,2008-05-03,7,101.0
2,100000004,1,3217,2008-05-07,3,102.0
3,100000004,5,3217,2008-05-09,6,NaN
4,100000001,7,3217,2008-05-10,7,101.0
5,100000004,4,3217,2008-05-13,3,103.0


From the nurse table, find those nurses who are yet to be registered. Return all the fields of nurse table.

In [19]:
  query = '''SELECT * FROM Nurse
             WHERE Registered = 0'''
data = pd.read_sql(query, con)
data

,EmployeeID,Name,Position,Registered,SSN
0,103,Paul Flowers,Nurse,0,333333330


From the nurse table, find the nurse who is the head of their department, return Nurse Name as “name”, Position as “Position”.

In [20]:
query = '''SELECT name as 'Nurse Name', position as 'Position' FROM Nurse
           WHERE Position = 'Head Nurse' '''
data = pd.read_sql(query, con)
data

,Nurse Name,Position
0,Carla Espinosa,Head Nurse


From the department and physician tables, find those physicians who are the head of the department, return Department name as “Department” and Physician name as “Physician”.

In [21]:
query = '''SELECT Physician.Name AS Physician, Department.Name AS Department
           FROM Physician
           JOIN Department
           ON Physician.EmployeeID = Department.Head '''
data = pd.read_sql(query, con)
data

,Physician,Department
0,Percival Cox,General Medicine
1,John Wen,Surgery
2,Molly Clock,Psychiatry


From the appointment table, count the number of patients who booked an appointment with at least one Physician, return count as “Number of patients taken at least one appointment”.

In [22]:
query = '''SELECT Patient, COUNT(Physician) AS 'Number of doctors that patients appointed'
           FROM Appointment
           GROUP BY Patient '''
data = pd.read_sql(query, con)
data

,Patient,Number of doctors that patients appointed
0,100000001,3
1,100000002,2
2,100000003,1
3,100000004,3


In [23]:
query = '''SELECT COUNT(DISTINCT(Patient)) AS 'Number of patients taken at least one appointment'
           FROM Appointment '''
data = pd.read_sql(query, con)
data

,Number of patients taken at least one appointment
0,4


From the room table, find the floor and block where the room number 212 belongs, eturn block floor as “Floor” and block code as “Block”.

In [24]:
query = '''SELECT BlockFloor AS 'Floor', BlockCode AS 'Block'
           FROM Room           
           WHERE Number = 212 '''
data = pd.read_sql(query, con)
data

,Floor,Block
0,2,2


From the room table, count the number available rooms, return count as “Number of available rooms”.

In [25]:
query = '''SELECT COUNT(Unavailable) AS 'Number of available rooms'
           FROM Room
           WHERE Unavailable = 0 '''
data = pd.read_sql(query, con)
data

,Number of available rooms
0,29


From the physician, affiliated_with, and department tables, find the physician and the departments they are affiliated with, return Physician name as “Physician”, and department name as “Department”.

In [26]:
query = '''SELECT *
           FROM Physician
           JOIN Affiliated_With
           ON Physician.EmployeeID = Affiliated_With.Physician
           JOIN Department
           ON Affiliated_With.Department = Department.DepartmentID'''
data = pd.read_sql(query, con)
data

,EmployeeID,Name,Position,SSN,Physician,Department,PrimaryAffiliation,DepartmentID,Name,Head
0,1,John Dorian,Staff Internist,111111111,1,1,1,1,General Medicine,4
1,2,Elliot Reid,Attending Physician,222222222,2,1,1,1,General Medicine,4
2,3,Christopher Turk,Surgical Attending Physician,333333333,3,1,0,1,General Medicine,4
3,3,Christopher Turk,Surgical Attending Physician,333333333,3,2,1,2,Surgery,7
4,4,Percival Cox,Senior Attending Physician,444444444,4,1,1,1,General Medicine,4
5,5,Bob Kelso,Head Chief of Medicine,555555555,5,1,1,1,General Medicine,4
6,6,Todd Quinlan,Surgical Attending Physician,666666666,6,2,1,2,Surgery,7
7,7,John Wen,Surgical Attending Physician,777777777,7,1,0,1,General Medicine,4
8,7,John Wen,Surgical Attending Physician,777777777,7,2,1,2,Surgery,7
9,8,Keith Dudemeister,MD Resident,888888888,8,1,1,1,General Medicine,4


In [27]:
query = '''SELECT Physician.Name AS Physician, Department.Name AS Department
           FROM Physician
           JOIN Affiliated_With
           ON Physician.EmployeeID = Affiliated_With.Physician
           JOIN Department
           ON Affiliated_With.Department = Department.DepartmentID
           WHERE PrimaryAffiliation = 1 '''
data = pd.read_sql(query, con)
data

,Physician,Department
0,John Dorian,General Medicine
1,Elliot Reid,General Medicine
2,Christopher Turk,Surgery
3,Percival Cox,General Medicine
4,Bob Kelso,General Medicine
5,Todd Quinlan,Surgery
6,John Wen,Surgery
7,Keith Dudemeister,General Medicine
8,Molly Clock,Psychiatry


From the physician, trained_in, and procedure tables, find those physicians who have trained for special treatment, return Physician name as “Physician”, treatment procedure name as “Treatment”.

In [28]:
query = '''SELECT Physician.Name AS Physician, Procedure.Name AS Procedure
           FROM Physician
           JOIN Trained_In
           ON Physician.EmployeeID = Trained_In.Physician
           JOIN Procedure
           ON Procedure.Code = Trained_In.Treatment '''
data = pd.read_sql(query, con)
data

,Physician,Procedure
0,Christopher Turk,Reverse Rhinopodoplasty
1,Christopher Turk,Obtuse Pyloric Recombobulation
2,Christopher Turk,Obfuscated Dermogastrotomy
3,Christopher Turk,Reversible Pancreomyoplasty
4,Christopher Turk,Follicular Demiectomy
5,Todd Quinlan,Obtuse Pyloric Recombobulation
6,Todd Quinlan,Obfuscated Dermogastrotomy
7,Todd Quinlan,Reversible Pancreomyoplasty
8,John Wen,Reverse Rhinopodoplasty
9,John Wen,Obtuse Pyloric Recombobulation


From the physician, affiliated_with, and department tables, find those physicians who are yet to be affiliated, return Physician name as “Physician”, Position, and department as “Department”.

In [29]:
query = '''SELECT *
           FROM Physician
           JOIN Affiliated_With
           ON Physician.EmployeeID = Affiliated_With.Physician
           JOIN Department
           ON Affiliated_With.Department = Department.DepartmentID '''
data = pd.read_sql(query, con)
data

,EmployeeID,Name,Position,SSN,Physician,Department,PrimaryAffiliation,DepartmentID,Name,Head
0,1,John Dorian,Staff Internist,111111111,1,1,1,1,General Medicine,4
1,2,Elliot Reid,Attending Physician,222222222,2,1,1,1,General Medicine,4
2,3,Christopher Turk,Surgical Attending Physician,333333333,3,1,0,1,General Medicine,4
3,3,Christopher Turk,Surgical Attending Physician,333333333,3,2,1,2,Surgery,7
4,4,Percival Cox,Senior Attending Physician,444444444,4,1,1,1,General Medicine,4
5,5,Bob Kelso,Head Chief of Medicine,555555555,5,1,1,1,General Medicine,4
6,6,Todd Quinlan,Surgical Attending Physician,666666666,6,2,1,2,Surgery,7
7,7,John Wen,Surgical Attending Physician,777777777,7,1,0,1,General Medicine,4
8,7,John Wen,Surgical Attending Physician,777777777,7,2,1,2,Surgery,7
9,8,Keith Dudemeister,MD Resident,888888888,8,1,1,1,General Medicine,4


In [30]:
query = '''SELECT Physician.Name AS Physician, Physician.Position AS Position, Department.Name AS Department
           FROM Physician
           JOIN Affiliated_With
           ON Physician.EmployeeID = Affiliated_With.Physician
           JOIN Department
           ON Affiliated_With.Department = Department.DepartmentID
           WHERE Affiliated_With.PrimaryAffiliation = 0 '''
data = pd.read_sql(query, con)
data

,Physician,Position,Department
0,Christopher Turk,Surgical Attending Physician,General Medicine
1,John Wen,Surgical Attending Physician,General Medicine


From the physician and trained_in tables, find those physicians who are not a specialized physician, return Physician name as “Physician”, position as “Designation”.

In [31]:
query = '''SELECT Physician.Name AS Physician, Physician.Position AS Designation
           FROM Physician
           LEFT JOIN Trained_In
           ON Physician.EmployeeID = Trained_In.Physician
           WHERE Treatment IS NULL '''
data = pd.read_sql(query, con)
data

,Physician,Designation
0,John Dorian,Staff Internist
1,Elliot Reid,Attending Physician
2,Percival Cox,Senior Attending Physician
3,Bob Kelso,Head Chief of Medicine
4,Keith Dudemeister,MD Resident
5,Molly Clock,Attending Psychiatrist


Or:

In [32]:
query = '''SELECT Physician.Name AS Physician, Physician.Position AS Designation
           FROM Physician
           WHERE EmployeeID NOT IN (SELECT Physician FROM Trained_In) '''
data = pd.read_sql(query, con)
data

,Physician,Designation
0,John Dorian,Staff Internist
1,Elliot Reid,Attending Physician
2,Percival Cox,Senior Attending Physician
3,Bob Kelso,Head Chief of Medicine
4,Keith Dudemeister,MD Resident
5,Molly Clock,Attending Psychiatrist


From the patient and physician tables, find the patients with their physicians by whom they got their preliminary treatment, return Patient name as “Patient”, address as “Address” and Physician name as “Physician”.

In [33]:
query = '''SELECT Physician.Name AS Physician, Patient.Name AS Patient, Patient.Address AS Address
           FROM Physician
           JOIN Patient
           ON Physician.EmployeeID = Patient.PCP '''
data = pd.read_sql(query, con)
data

,Physician,Patient,Address
0,John Dorian,John Smith,42 Foobar Lane
1,Elliot Reid,Grace Ritchie,37 Snafu Drive
2,Elliot Reid,Random J. Patient,101 Omgbbq Street
3,Christopher Turk,Dennis Doe,1100 Foobaz Avenue


From the patient and appointment tables, find the patients and the number of physicians they have taken appointment, return Patient name as “Patient”, number of Physicians as “Appointment for No. of Physicians”.

In [34]:
query = '''SELECT Patient.Name AS Patient, COUNT(Appointment.Physician) AS 'Appointment for No. of Physicians'
           FROM Patient
           JOIN Appointment
           ON Patient.SSN = Appointment.Patient
           GROUP BY Patient.SSN '''
data = pd.read_sql(query, con)
data

,Patient,Appointment for No. of Physicians
0,John Smith,3
1,Grace Ritchie,2
2,Random J. Patient,1
3,Dennis Doe,3


From the appointment table, count number of unique patients who got an appointment for examination room ‘C’, return unique patients as “No. of patients got appointment for room C”.

In [35]:
query = '''SELECT COUNT(DISTINCT(Patient)) AS 'No. of patients got appointment for room C'
           FROM Appointment
           WHERE ExaminationRoom = 'C' '''
data = pd.read_sql(query, con)
data

,No. of patients got appointment for room C
0,3


From the patient and appointment tables, find the name of the patients and the number of the room where they have to go for their treatment, return patient name as “Patient”, examination room as “Room No.”, and starting date time as Date “Date and Time of appointment”.

In [36]:
query = '''SELECT Patient.Name AS 'Patient', Appointment.ExaminationRoom AS 'Room No.', Appointment.Start AS 'Date and Time of appointment'
           FROM Patient
           JOIN Appointment
           ON Patient.SSN = Appointment.Patient '''
data = pd.read_sql(query, con)
data

,Patient,Room No.,Date and Time of appointment
0,John Smith,A,2008-04-24 10:00
1,Grace Ritchie,B,2008-04-24 10:00
2,John Smith,A,2008-04-25 10:00
3,Dennis Doe,B,2008-04-25 10:00
4,Dennis Doe,C,2008-04-26 10:00
5,Random J. Patient,C,2008-04-26 11:00
6,John Smith,C,2008-04-26 12:00
7,Dennis Doe,A,2008-04-27 10:00
8,Grace Ritchie,B,2008-04-27 10:00


From the nurse and appointment tables, find the name of the nurses and the room scheduled, where they will assist the physicians, return Nurse Name as “Name of the Nurse” and examination room as “Room No.”.

In [37]:
query = '''SELECT Nurse.Name AS 'Name of the Nurse', Appointment.ExaminationRoom AS 'Room No.'
           FROM Nurse
           JOIN Appointment
           ON Nurse.EmployeeID = Appointment.PrepNurse '''
data = pd.read_sql(query, con)
data

,Name of the Nurse,Room No.
0,Carla Espinosa,A
1,Carla Espinosa,B
2,Laverne Roberts,A
3,Paul Flowers,B
4,Paul Flowers,C
5,Laverne Roberts,A
6,Carla Espinosa,B


From the patient, appointment, nurse, and physician tables, find those patients who taken the appointment on the 25th of April at 10 am, return Name of the patient, Name of the Nurse assisting the physician, Physician Name as “Name of the physician”, examination room as “Room No.”, schedule date and approximate time to meet the physician.

In [38]:
query = '''SELECT Patient.Name AS 'Patient Name', Nurse.Name AS 'Name of the nurse', Physician.Name AS 'Name of the physician', Appointment.ExaminationRoom AS 'Room No.'
           FROM Physician
           JOIN Appointment
           ON Physician.EmployeeID = Appointment.Physician
           JOIN Patient
           ON Appointment.Patient = Patient.SSN
           JOIN Nurse
           ON Nurse.EmployeeID = Appointment.PrepNurse
           WHERE Appointment.Start = '2008-04-25 10:00' '''
data = pd.read_sql(query, con)
data

,Patient Name,Name of the nurse,Name of the physician,Room No.
0,John Smith,Laverne Roberts,John Dorian,A
1,Dennis Doe,Paul Flowers,Percival Cox,B


From the patient, appointment, and physician tables, find those patients and their physicians who do not require any assistance of a nurse, return Name of the patient as “Name of the patient”, Name of the Physician as “Name of the physician” and examination room as “Room No.”.

In [39]:
query = '''SELECT Patient.Name AS 'Name of the patient', Physician.Name AS 'Name of the physician', Appointment.ExaminationRoom AS 'Room No.'
           FROM Physician
           JOIN Appointment
           ON Physician.EmployeeID = Appointment.Physician
           JOIN Patient
           ON Appointment.Patient = Patient.SSN
           WHERE Appointment.PrepNurse IS NULL '''
data = pd.read_sql(query, con)
data

,Name of the patient,Name of the physician,Room No.
0,Dennis Doe,Percival Cox,C
1,John Smith,Christopher Turk,C


From the patient, prescribes, physician, and medication tables, find the patients and their treating physicians and medication, return Patient name as “Patient”, Physician name as “Physician”, Medication name as “Medication”.

In [40]:
query = '''SELECT Patient.Name AS 'Name of the patient', Physician.Name AS 'Name of the physician', Medication.Name AS 'Medication'
           FROM Patient
           JOIN Prescribes
           ON Patient.SSN = Prescribes.Patient
           JOIN Physician
           ON Prescribes.Physician = Physician.EmployeeID
           JOIN Medication
           ON Prescribes.Medication = Medication.Code '''
data = pd.read_sql(query, con)
data

,Name of the patient,Name of the physician,Medication
0,John Smith,John Dorian,Procrastin-X
1,Dennis Doe,Molly Clock,Thesisin
2,Dennis Doe,Molly Clock,Thesisin


From the patient, prescribes, physician, and medication tables, find those patients who have taken an advanced appointment. Return Patient name as “Patient”, Physician name as “Physician” and Medication name as “Medication”.

In [41]:
query = '''SELECT Patient.Name AS 'Patient', Physician.Name AS 'Physician', Medication.Name AS 'Medication'
           FROM Patient
           JOIN Prescribes
           ON Patient.SSN = Prescribes.Patient
           JOIN Physician
           ON Prescribes.Physician = Physician.EmployeeID
           JOIN Medication
           ON Prescribes.Medication = Medication.Code
           WHERE Prescribes.Appointment IS NOT NULL '''
data = pd.read_sql(query, con)
data

,Patient,Physician,Medication
0,John Smith,John Dorian,Procrastin-X
1,Dennis Doe,Molly Clock,Thesisin


 From the patient, prescribes, physician, and medication tables, find those patients who did not take any appointment, return Patient name as “Patient”, Physician name as “Physician” and Medication name as “Medication”.

In [42]:
query = '''SELECT Patient.Name AS 'Patient', Physician.Name AS 'Physician', Medication.Name AS 'Medication'
           FROM Patient
           JOIN Prescribes
           ON Patient.SSN = Prescribes.Patient
           JOIN Physician
           ON Prescribes.Physician = Physician.EmployeeID
           JOIN Medication
           ON Prescribes.Medication = Medication.Code
           WHERE Prescribes.Appointment IS NULL '''
data = pd.read_sql(query, con)
data

,Patient,Physician,Medication
0,Dennis Doe,Molly Clock,Thesisin


From the room table, count the number of available rooms in each block, sort the result-set on ID of the block. Return ID of the block as “Block”, count number of available rooms as “Number of available rooms”.

In [43]:
query = '''SELECT BlockCode AS Block, COUNT(BlockCode) AS 'Number of available rooms'
           FROM Room
           WHERE Unavailable = 0
           GROUP BY BlockCode
           ORDER BY BlockCode '''
data = pd.read_sql(query, con)
data

,Block,Number of available rooms
0,1,9
1,2,10
2,3,10


From the room table, count the number of available rooms in each floor, sort the result-set on block floor. Return floor ID as “Floor” and count the number of available rooms as “Number of available rooms”.

In [44]:
query = '''SELECT BlockFloor AS Floor, COUNT(blockfloor) AS 'Number of available rooms'
           FROM Room
           WHERE Unavailable = 0
           GROUP BY BlockFloor
           ORDER BY BlockFloor '''
data = pd.read_sql(query, con)
data

,Floor,Number of available rooms
0,1,8
1,2,7
2,3,7
3,4,7


From the room table, count the number of available rooms for each floor in each block, sort the result-set on floor ID, ID of the block. Return the floor ID as “Floor”, ID of the block as “Block”, and number of available rooms as “Number of available rooms”.

In [45]:
query = '''SELECT BlockFloor AS 'Floor', BlockCode AS 'Block', COUNT(Unavailable) AS 'Number of available rooms'
           FROM Room
           WHERE Unavailable = 0
           GROUP BY BlockFloor, BlockCode
           ORDER BY BlockFloor, BlockCode '''
data = pd.read_sql(query, con)
data

,Floor,Block,Number of available rooms
0,1,1,3
1,1,2,2
2,1,3,3
3,2,1,2
4,2,2,2
5,2,3,3
6,3,1,2
7,3,2,3
8,3,3,2
9,4,1,2


From the room table, count the number of unavailable rooms for each block in each floor. Sort the result-set on block floor, block code. Return the floor ID as “Floor”, block ID as “Block”, and number of unavailable as “Number of unavailable rooms”.

In [46]:
query = '''SELECT BlockFloor AS 'Floor', BlockCode AS 'Block', COUNT(Unavailable) AS 'Number of available rooms'
           FROM Room
           WHERE Unavailable = 1
           GROUP BY BlockFloor, BlockCode
           ORDER BY BlockFloor, BlockCode '''
data = pd.read_sql(query, con)
data

,Floor,Block,Number of available rooms
0,1,2,1
1,2,1,1
2,2,2,1
3,3,1,1
4,3,3,1
5,4,1,1
6,4,3,1


From the room table, find the floor where the maximum numbers of rooms are available. Return floor ID as “Floor”, count “Number of available rooms”.

In [47]:
query = '''SELECT BlockFloor AS 'Floor', COUNT(Unavailable) AS 'Number of available rooms'
           FROM Room
           WHERE Unavailable = 0
           GROUP BY BlockFloor
           ORDER BY 'Number of available rooms' DESC LIMIT 1 '''
data = pd.read_sql(query, con)
data

,Floor,Number of available rooms
0,1,8


From the patient, stay, and room tables, find the name of the patients, their block, floor, and room number where they admitted.

In [57]:
query = '''SELECT Patient.Name, Room.BlockCode, Room.BlockFloor, Stay.Room
           FROM Patient
           JOIN Stay
           ON Patient.SSN = Stay.Patient
           JOIN Room
           ON Stay.Room = Room.Number '''
data = pd.read_sql(query, con)
data

,Name,BlockCode,BlockFloor,Room
0,John Smith,2,1,111
1,Random J. Patient,3,1,123
2,Dennis Doe,2,1,112


From the nurse and on_call tables, find the nurses and the block where they are booked for attending the patients on call. Return Nurse Name as “Nurse”, Block code as “Block”.

In [59]:
query = '''SELECT Nurse.Name AS Nurse, BlockCode AS Block
           FROM Nurse
           JOIN On_Call
           ON Nurse.EmployeeID = On_Call.Nurse '''
data = pd.read_sql(query, con)
data

,Nurse,Block
0,Carla Espinosa,1
1,Carla Espinosa,2
2,Laverne Roberts,3
3,Paul Flowers,1
4,Paul Flowers,2
5,Paul Flowers,3


From the patient, undergoes, physician, nurse, stay, and room tables, get
a) name of the patient,
b) name of the physician who is treating him or her,
c) name of the nurse who is attending him or her,
d) the date of release,
e) in which room the patient has admitted and which floor and block the room belongs to respectively.

In [65]:
query = '''SELECT Patient.Name, Physician.Name, Nurse.Name, Stay.End, Room.Number, Room.BlockFloor, Room.BlockCode
           FROM Patient
           JOIN Undergoes
           ON Patient.SSN = Undergoes.Patient
           JOIN Physician
           ON Undergoes.Physician = Physician.EmployeeID
           JOIN Nurse
           ON Undergoes.AssistingNurse = Nurse.EmployeeID
           JOIN Stay
           ON Undergoes.Stay = Stay.StayID
           JOIN Room ON Stay.Room = Room.Number '''
data = pd.read_sql(query, con)
data

,Name,Name,Name,End,Number,BlockFloor,BlockCode
0,John Smith,Christopher Turk,Carla Espinosa,2008-05-04,111,1,2
1,John Smith,John Wen,Carla Espinosa,2008-05-04,111,1,2
2,Dennis Doe,Christopher Turk,Laverne Roberts,2008-05-03,112,1,2
3,John Smith,John Wen,Carla Espinosa,2008-05-03,112,1,2
4,Dennis Doe,Christopher Turk,Paul Flowers,2008-05-03,112,1,2


From the patient, undergoes, trained_in, procedure, and patient tables, find all the physicians, their procedure, date when the procedure was carried out and name of the patient on which procedure have been carried out but those physicians are not certified for that procedure. Return Physician Name as “Physician”, Procedure Name as “Procedure”, date, and Patient. Name as “Patient”.

In [75]:
query = '''SELECT Physician.Name AS 'Physician', Procedure.Name AS 'Procedure', Undergoes.Date, Patient.Name AS 'Patient'
           FROM Physician
           JOIN Undergoes
           ON Physician.EmployeeID = Undergoes.Physician
           LEFT JOIN Trained_In
           ON Physician.EmployeeID = Trained_In.Physician
           AND Undergoes.procedure = Trained_In.Treatment          
           JOIN Procedure
           ON Undergoes.Procedure = Procedure.Code
           JOIN Patient
           ON Undergoes.Patient = Patient.SSN
           WHERE Trained_In.Treatment IS NULL '''
data = pd.read_sql(query, con)
data

,Physician,Procedure,Date,Patient
0,Christopher Turk,Complete Walletectomy,2008-05-13,Dennis Doe


From the nurse, on_call, and room tables, find those nurses who have ever been on call for room 122. Return name of the nurses.

In [88]:
query = '''SELECT DISTINCT(Nurse.Name)
           FROM Nurse
           JOIN On_Call
           ON Nurse.EmployeeID = On_Call.Nurse
           JOIN Room
           ON On_Call.BlockFloor = Room.BlockFloor           
           WHERE Room.Number = 122
           '''
data = pd.read_sql(query, con)
data

,Name
0,Carla Espinosa
1,Laverne Roberts
2,Paul Flowers


From the patient, prescribes, and physician tables, find those patients who have been prescribed by some medication by his/her physician who has carried out primary care. Return Patient name as “Patient”, and Physician Name as “Physician”.

In [91]:
query = '''SELECT Patient.Name AS 'Patient', Physician.Name AS 'Physician'
           FROM Patient
           JOIN Prescribes
           ON Patient.SSN = Prescribes.Patient
           JOIN Physician
           ON Prescribes.Physician = Physician.EmployeeID           
           WHERE Patient.PCP = Physician.EmployeeID
           '''
data = pd.read_sql(query, con)
data

,Patient,Physician
0,John Smith,John Dorian


From the patient, physician, undergoes, and procedure tables, find those patients who have been undergone a procedure costing more than $5,000 and the name of that physician who has carried out primary care. Return name of the patient as “Patient”, name of the physician as “Primary Physician”, and cost for the procedure as “Procedure Cost”.

In [92]:
query = '''SELECT Patient.Name AS 'Patient', Physician.Name AS 'Physician', Procedure.Cost AS 'Procedure Cost'
           FROM Patient
           JOIN Physician
           ON Patient.PCP = Physician.EmployeeID
           JOIN Undergoes
           ON Patient.SSN = Undergoes.Patient
           JOIN Procedure
           ON Undergoes.Procedure = Procedure.Code           
           WHERE Procedure.Cost > 5000 '''
data = pd.read_sql(query, con)
data

,Patient,Physician,Procedure Cost
0,John Smith,John Dorian,5600.0
1,Dennis Doe,Christopher Turk,10000.0
